# 실습3: 멜론 Top100 긁기
### 1. 앨범 Title 긁기
### 2. 앨범 Artist 긁기
### 3. 앨범 이름 긁기
### 4. 좋아요 수 긁기 (Rest API 활용)
#### 참고: https://www.melon.com/chart/

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd

In [5]:
target_url = 'https://www.melon.com/chart/'

# 대부분의 웹사이트는 비정상적인 접근을 허용하지 않는다.
# 유저가 웹 브라우저를 통해 접근한 방식을 제외하고는 대부분 비정상적인 접근이다.(크롤링 포함)
# 따라서, 내가 누구인지, 어떻게 너희들의 웹페이지에 들어왔는지를 알려주는 정보가 필요하다.
# 이 역할을 하는 것이 header.
# header에 포함되는 대표적인 요소로 user-agent가 있다. -> 내가 어떤 웹브라우저를 통해 들어왔는지 알려주는 것.
# 개발자 탭에서 Network 탭의 Fetch/XHR을 보면 user-agent가 나와있다.
# 단순히 웹사이트를 속이기 위함이기 때문에 어떤 user-agent를 사용하든 상관없다. 즉, 구글링을 한 다음 돌아다니는 user-agent를 사용해도 괜찮다.
header = {
    'User-Agent':'ozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}

res = req.get(target_url, headers=header)
res

<Response [200]>

In [69]:
soup = bs(res.text, 'html.parser')

#### 여기서부터 핵심 ####
# 크롤링의 핵심은 아래와 같다.
# (1) 데이터를 어떻게 구성할 것인가?
# (2) 어떤 태그로 접근할 것인가?

# (1) 데이터를 어떻게 구성할 것인가?
# 데이터를 어떻게 구성할 지 미리 계획을 짜는 것은 크롤링을 함에 있어서 가장 중요한 단계이다.
# 데이터를 어떻게 구성할 지 계획 짜는 것은 내가 지금 보고있는 웹사이트에서 컬럼을 어떻게 구성할 것인지, 머릿속으로 엑셀 시트를 상상하는 것이다.
# 예를 들면, 본 실습의 경우, 앨범 Title, Artist, 앨범명이 컬럼으로 들어갈 것이다.

# 멜론에서 친절하게 Table형태로 보여주어서 우리는 데이터를 어떻게 구성할 지 쉽게 떠올릴 수 있지만, 
# 모든 웹사이트가 이렇게 친절하게 데이터 형태를 구성해주지는 않는다. 따라서 직접 상상을 통해 미리 계획을 짜두어야 한다.

# (2) 어떤 태그로 접근할 것인가?
# 가장 어려운 단계.
# 태그를 찾기란 쉽지 않다. 너무 broad하지 않고, 그렇다고 또 너무 deep하지 않은 중간 부분의 태그를 찾아야 한다.
# 가장 좋은 코드는 규칙성이 한 눈에 보이고 일반화가 가능한 코드이다. 

song_list = soup.select('tbody > tr')
song_list[0].select('.wrap_song_info > div > span')[0].text.replace("\n", "")
song_list[0].select('.wrap_song_info > div > span')[1].text.replace("\n", "")
song_list[0].select('.wrap_song_info > div > a')[1].text.replace("\n", "")

# 윗부분처럼 이것저것 태그를 하나하나 시도해보고, 너무 broad하지 않고, 너무 deep하지도 않은 중간선을 찾았다는 생각이 들 때
# 코드를 일반화하여 for문을 반복해준다.
title = [row.select('.wrap_song_info > div > span')[0].text.replace("\n", "") for row in song_list]
artist = [row.select('.wrap_song_info > div > span')[1].text.replace("\n", "") for row in song_list]
album = [row.select('.wrap_song_info > div > a')[1].text.replace("\n", "") for row in song_list]

df = pd.DataFrame({
    'title':title,
    'artist':artist,
    'album':album
})

df.head()

,title,artist,album
0,새삥 (Prod. ZICO) (Feat. 호미들),지코 (ZICO),스트릿 맨 파이터(SMF) Original Vol.3 (계급미션)
1,After LIKE,IVE (아이브),After LIKE
2,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans'
3,Shut Down,BLACKPINK,BORN PINK
4,Rush Hour (Feat. j-hope of BTS),Crush,Rush Hour


In [82]:
# 크롤링은 웹페이지에서 눈에 보이는 부분을 긁어올 수도 있지만...

# Backend에서 Frontend로 데이터가 전송되는 중간 부분을 Rest API를 통해 긁어올 수도 있다. 
# 이 경우에는 눈에 보이지 않는 부분을 가져올 수 있을 뿐만 아니라,
# 동적 웹페이지까지 가져올 수 있다.

# 멜론 Top100의 좋아요 수를 html 태그를 사용해서 가져와보자. 
# 아마 우리 눈에 보이는 좋아요 수와는 다른 숫자가 긁힐 것이다. -> 페이지가 동적으로 구성되어있기 때문

# 이 경우, Rest API를 활용해서 좋아요 수를 가져올 수 있다.
# 개발자도구 -> Network탭 -> Fetch/XHR을 클릭해보자
# (1) Response를 눌러서 우리가 받을 정보가 어떤 것인지 확인한다. (일일이 하나하나 확인해야 한다. 딕셔너리 형태로 생겼다.)
# (2) 만약 찾았으면 Header 및 Payload를 눌러서 요청을 보낼 때 필요한 정보를 확인한다. (request url, parameter, user-agent 등등)


tmp_url = 'https://www.melon.com/commonlike/getSongLike.json?contsIds=35595136%2C35546497%2C35454426%2C35640077%2C35665282%2C35454425%2C35542908%2C34847378%2C35668899%2C35643794%2C34061322%2C35685439%2C35383397%2C34908740%2C35669445%2C35485544%2C35361345%2C35008524%2C35504734%2C35595137%2C35383398%2C34657844%2C35252996%2C34754292%2C34927767%2C35008525%2C32508053%2C34787226%2C34943312%2C35008527%2C35413033%2C34997078%2C34864406%2C35008528%2C34349913%2C35008526%2C34100776%2C35009233%2C35008531%2C35008534%2C34431086%2C4446485%2C35640751%2C34819473%2C34875621%2C35008532%2C33496587%2C35333345%2C35008529%2C34930368%2C35238221%2C35008530%2C35008533%2C32872978%2C35331586%2C35484543%2C34845949%2C33658563%2C35505810%2C35526685%2C30962526%2C35391594%2C33061995%2C33480898%2C33507137%2C35238220%2C34772475%2C34101563%2C35560680%2C35643842%2C34626109%2C34481680%2C35388184%2C3414749%2C35272060%2C31666417%2C30244931%2C35177030%2C35215215%2C33487342%2C34752700%2C33666269%2C33359309%2C35667692%2C34701627%2C34754299%2C34817660%2C33978183%2C33655994%2C34538515%2C35353251%2C35640848%2C35709043%2C34256568%2C33239419%2C35609035%2C35711701%2C35690912%2C32698101%2C35694868'
res2 = req.get(tmp_url, headers=header)
res2.text

'{"contsLike":[{"CONTSID":35595136,"LIKEYN":"N","SUMMCNT":101004},{"CONTSID":35546497,"LIKEYN":"N","SUMMCNT":176554},{"CONTSID":35454426,"LIKEYN":"N","SUMMCNT":144325},{"CONTSID":35640077,"LIKEYN":"N","SUMMCNT":89305},{"CONTSID":35665282,"LIKEYN":"N","SUMMCNT":58262},{"CONTSID":35454425,"LIKEYN":"N","SUMMCNT":170850},{"CONTSID":35542908,"LIKEYN":"N","SUMMCNT":139793},{"CONTSID":34847378,"LIKEYN":"N","SUMMCNT":205850},{"CONTSID":35668899,"LIKEYN":"N","SUMMCNT":61753},{"CONTSID":35643794,"LIKEYN":"N","SUMMCNT":29608},{"CONTSID":34061322,"LIKEYN":"N","SUMMCNT":177894},{"CONTSID":35685439,"LIKEYN":"N","SUMMCNT":31307},{"CONTSID":35383397,"LIKEYN":"N","SUMMCNT":116792},{"CONTSID":34908740,"LIKEYN":"N","SUMMCNT":78415},{"CONTSID":35669445,"LIKEYN":"N","SUMMCNT":35667},{"CONTSID":35485544,"LIKEYN":"N","SUMMCNT":88899},{"CONTSID":35361345,"LIKEYN":"N","SUMMCNT":107315},{"CONTSID":35008524,"LIKEYN":"N","SUMMCNT":62129},{"CONTSID":35504734,"LIKEYN":"N","SUMMCNT":97412},{"CONTSID":35595137,"LIKEY

In [88]:
# Rest API를 통해 우리가 가져온 데이터는 딕셔너리가 아니라 JSON이라고 부른다.
# 딕셔너리는 파이썬에서 사용되는 자료구조.
# JSON은 서버 - 클라이언트 간 데이터 송수신 규격
# 하지만, 형태는 유사하다.

# 이전 실습에서 우리는 html 태그를 찾기 전, text를 html로 파싱해야 한다고 배웠다.
# 사실 엄밀히 따지면, html문서를 파이썬에서 읽을 수 있도록 파싱하는 것이다.

# 마찬가지로, Rest API로 받은 json 데이터를 파이썬에서 읽을 수 있도록 해야 한다.
# json.loads() 명령어는 json 데이터를 파이썬에서 읽을 수 있게 해준다.
# 파싱만 해주면 그 다음 단계는 매우 수월하다는 점에서 html 태그로 찾는 크롤링 방식보다 훨씬 간편하다고 할 수 있다.
import json
import datetime


j = json.loads(res2.text)
likes = [i['SUMMCNT'] for i in j['contsLike']]
likes

[101004,
 176554,
 144325,
 89305,
 58262,
 170850,
 139793,
 205850,
 61753,
 29608,
 177894,
 31307,
 116792,
 78415,
 35667,
 88899,
 107315,
 62129,
 97412,
 32502,
 116763,
 161304,
 45722,
 216644,
 183687,
 51239,
 160308,
 74521,
 125993,
 43640,
 71081,
 123148,
 120924,
 42074,
 161455,
 40985,
 74210,
 79269,
 41684,
 41182,
 146273,
 236359,
 46533,
 50818,
 119725,
 40126,
 153515,
 85821,
 39675,
 94213,
 73482,
 39375,
 39430,
 441454,
 91826,
 33095,
 284871,
 237741,
 45514,
 31018,
 405833,
 37592,
 205629,
 290437,
 278046,
 62809,
 141463,
 210241,
 35905,
 33093,
 128172,
 163142,
 28303,
 201833,
 115603,
 345179,
 551376,
 50286,
 40261,
 252696,
 124892,
 213460,
 140989,
 13615,
 84555,
 77456,
 76839,
 141070,
 186164,
 132714,
 37752,
 47202,
 4076,
 214356,
 326147,
 14003,
 9567,
 12519,
 196764,
 7555]